In [36]:
import pandas as pd
import os
from google.colab import files

repo_name = "PVRecycling"
repo_url = "https://github.com/pleu/PVRecycling.git"
branch_name = "main"

if os.path.basename(os.getcwd()) == repo_name:
  os.chdir("/content")

if os.path.exists(repo_name):
  !rm -rf {repo_name}

os.system(f"git clone --branch {branch_name} {repo_url}")
os.chdir(repo_name)

In [37]:

def calculate_production_step_data(input_file='data/manufacturing_values.csv',
                                 output_file='data/production_step_data.csv',
                                 years=range(2021, 2026)):

    input_df = pd.read_csv(input_file)
    output_columns = ['Year', 'Production Step', 'Unit', 'Distribution', 'Process Cost', 'Process Cost Min', 'Process Cost Max']

    # Read existing data
    existing_df = pd.read_csv(output_file)
    existing_2020 = existing_df[existing_df['Year'] == 2020].copy()

    all_years_data = []

    for year in years:
        # Create output structure for current year
        data = {
            'Production Step': [
                'Purchase mg-Si',
                'SoG Poly-Si (exclude mg-Si cost)',
                'Mono Wafer (exclude poly-Si cost)',
                'Mono PERC cell (exclude wafer cost)',
                'Mono PERC module (exclude cell cost)',
                'Recycled mg-Si',
                'Recycled SOG poly-Si',
                'Recycled Mono wafer'],
            'Unit': ['kg', 'kg', 'wafer', 'cell', 'module', 'kg', 'kg', 'wafer'],
            'Distribution': ['Log-normal'] * 5 + ['Uniform'] * 3,
            'Year': [year] * 8}

        # Create dataframe for current year
        year_df = pd.DataFrame(data)

        # Get manufacturing values for this year
        year_values = input_df.set_index('variable')[str(year)].astype(float)

        # Set process costs
        year_df.loc[0, 'Process Cost'] = year_values['mg-SI ($/kg)']
        year_df.loc[1, 'Process Cost'] = year_values['poly-Si ($/kg)']
        year_df.loc[2, 'Process Cost'] = year_values['mono-wafer ($/wafer)']
        year_df.loc[3, 'Process Cost'] = year_values['mono-cell ($/cell)']
        year_df.loc[4, 'Process Cost'] = year_values['mono-module ($/module)']

        # Set recycled material costs (50% of virgin materials)
        year_df.loc[5, 'Process Cost'] = 0.5 * year_values['mg-SI ($/kg)']
        year_df.loc[6, 'Process Cost'] = 0.5 * year_values['poly-Si ($/kg)']
        year_df.loc[7, 'Process Cost'] = 0.5 * year_values['mono-wafer ($/wafer)']

        # Calculate min/max values
        for i in range(5):  # First 5 are Log-normal
            cost = year_df.loc[i, 'Process Cost']
            year_df.loc[i, 'Process Cost Min'] = 0.8 * cost
            year_df.loc[i, 'Process Cost Max'] = 1.2 * cost

        for i in range(5, 8):  # Last 3 are Uniform
            cost = year_df.loc[i, 'Process Cost']
            year_df.loc[i, 'Process Cost Min'] = 0
            year_df.loc[i, 'Process Cost Max'] = 2 * cost

        all_years_data.append(year_df)

    # Combine the data
    new_years_df = pd.concat(all_years_data, ignore_index=True)
    combined_df = pd.concat([existing_2020, new_years_df], ignore_index=True)
    combined_df = combined_df[output_columns]

    # Save the result
    combined_df.to_csv(output_file, index=False)

    return combined_df

# Run the function
production_data_df = calculate_production_step_data()
print(production_data_df.head())

# Download the file
files.download('data/production_step_data.csv')

   Year                       Production Step    Unit Distribution  \
0  2020                        Purchase mg-Si      kg   Log-normal   
1  2020      SoG Poly-Si (exclude mg-Si cost)      kg   Log-normal   
2  2020     Mono Wafer (exclude poly-Si cost)   wafer   Log-normal   
3  2020   Mono PERC cell (exclude wafer cost)    cell   Log-normal   
4  2020  Mono PERC module (exclude cell cost)  module   Log-normal   

   Process Cost  Process Cost Min  Process Cost Max  
0          2.50              2.00              3.00  
1          7.20              5.80              8.60  
2          0.24              0.19              0.29  
3          0.22              0.17              0.26  
4         39.00             31.00             47.00  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>